In [0]:
!git clone https://github.com/facebookresearch/ParlAI.git
!cd ParlAI; python setup.py develop
!pip install 'git+https://github.com/rsennrich/subword-nmt.git#egg=subword-nmt'
!pip install -U discord.py

In [0]:
import pexpect
class ChatHandler:
    def __init__(self):
        self.child = pexpect.spawn('python ParlAI/parlai/scripts/interactive.py -t blended_skill_talk -mf zoo:blender/blender_90M/model', timeout=None)
        self.child.expect('Enter Your Message:')
        self.personality = self.child.before.decode('utf-8', 'ignore').split('[context]')[1]
    def listen(self):
        response = self.child.before
        resp = response.split(b'1m')
        respfinal = resp[1].split(b'\x1b')
        return respfinal[0].decode('utf-8')
    def say(self, message):
        self.child.sendline(message)
        self.child.expect('Enter Your Message:')

In [0]:
import discord
TOKEN = "PUT YOUR TOKEN HERE"
SERVER = 'DiscordDrive'
CHANNEL_NAME = 'bot-conversation'
class Chattr1(discord.Client):
    async def on_ready(self):
        print("Prepping Chat")
        self.chat = ChatHandler()
        print("Chat Prepped")
        for guild in self.guilds:
            print(guild.name)
            if guild.name == SERVER:
                break
        for channel in guild.channels:
            if channel.name == CHANNEL_NAME:
                await channel.send('hi')
                print("Channel Found")
                break
    async def on_message(self, message):
        if message.channel.name == CHANNEL_NAME and message.channel.guild.name == SERVER and message.author.id != self.user.id:
            saying = message.content[7:]
            self.chat.say(saying)
            response = self.chat.listen()
            await message.channel.send(response)
client = Chattr1()
client.run(TOKEN)